<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/LSTM1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# download datasets
!wget 'https://raw.githubusercontent.com/donglee-afar/logdeep/master/data/hdfs/hdfs_train'
!wget 'https://raw.githubusercontent.com/donglee-afar/logdeep/master/data/hdfs/hdfs_test_normal'
!wget 'https://raw.githubusercontent.com/donglee-afar/logdeep/master/data/hdfs/hdfs_test_abnormal'

--2023-07-19 16:39:14--  https://raw.githubusercontent.com/donglee-afar/logdeep/master/data/hdfs/hdfs_train
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 257875 (252K) [text/plain]
Saving to: ‘hdfs_train’

hdfs_train          100%[===================>] 251.83K  --.-KB/s    in 0.03s   

2023-07-19 16:39:14 (7.85 MB/s) - ‘hdfs_train’ saved [257875/257875]

--2023-07-19 16:39:14--  https://raw.githubusercontent.com/donglee-afar/logdeep/master/data/hdfs/hdfs_test_normal
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29284282 (28M) 

In [25]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import TensorDataset, DataLoader
import os
import numpy as np

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
name = 'hdfs_train'
window_size = 10
num_sessions = 0
inputs = []
outputs = []

with open('/content/' + name, 'r') as f:
        for row in f:
            num_sessions += 1
            line = [int(i) - 1 for i in row.strip().split()] # we substract by one from templates index for starting from zero
            for i in range(len(line) - window_size):
                inputs.append(line[i:i + window_size])
                outputs.append(line[i + window_size])

print('Number of sessions({}): {}'.format(name, num_sessions))
print('Number of seqs({}): {}'.format(name, len(inputs)))
dataset = TensorDataset(torch.tensor(inputs, dtype=torch.float), torch.tensor(outputs))

Number of sessions(hdfs_test_abnormal): 16838
Number of seqs(hdfs_test_abnormal): 162697


In [4]:
print(inputs[15:18])
print(outputs[15:18])

[[4, 25, 25, 25, 10, 8, 10, 8, 10, 8], [25, 25, 25, 10, 8, 10, 8, 10, 8, 1], [25, 25, 10, 8, 10, 8, 10, 8, 1, 2]]
[1, 2, 22]


In [5]:
# unique outputs(templates) in training dataset
print(set(outputs))
len(set(outputs))

{1, 2, 3, 4, 5, 8, 10, 15, 17, 19, 20, 21, 22, 24, 25, 26, 27}


17

To represent a single template, we use a “one-hot vector” of
size <1 x n_templates>. A one-hot vector is filled with 0s
except for a 1 at index of the current template, e.g. "b" = <0 1 0 0 0 ...>.

In [4]:

n_templates = 28 #total number of unique templates

def template_to_tensor(template):
    tensor = [0] * n_templates
    if template != -1 :    # we put template= -1 for session less than windows_size in empty part and it will be all zero
      tensor[template] = 1
    return tensor

In [12]:
print(template_to_tensor(2))
print(len(template_to_tensor(2)))

[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
28


In [2]:
print(template_to_tensor(-1))
print(len(template_to_tensor(-1)))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
28


In [ ]:
name = 'hdfs_train'
window_size = 10
num_sessions = 0
inputs = []
outputs = []

with open('/content/' + name, 'r') as f:
        for row in f:
            num_sessions += 1
            line = [template_to_tensor(int(i) - 1) for i in row.strip().split()] # we substract by one from templates index for starting from zero
            for i in range(len(line) - window_size):
                inputs.append(line[i:i + window_size])
                outputs.append(line[i + window_size])

print('Number of sessions({}): {}'.format(name, num_sessions))
print('Number of seqs({}): {}'.format(name, len(inputs)))
dataset = TensorDataset(torch.tensor(inputs, dtype=torch.float), torch.tensor(outputs,dtype=torch.float ))

Exception ignored in: <function _xla_gc_callback at 0x7ca8d211c8b0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 103, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 
Exception ignored in: <function _xla_gc_callback at 0x7ca8d211c8b0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 103, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


In [13]:
print(inputs[15])
print(outputs[15])

[[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [14]:
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(Model, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))  # out.shape : [batch_size, sequence_length, hidden_size]
        out = self.fc(out[:, -1, :]) #The : before , -1, : indicates that we want to include all elements along the first dimension (batch dimension). -1 represents the index of the last element along the second dimension (sequence length). : after , -1 indicates that we want to include all elements along the third dimension (hidden size)
        return out

In [15]:
input_size = 28 # means each input has one feature(template's code)
num_layers = 2
hidden_size = 64
num_classes = 28
batch_size = 2048
num_epochs = 300
model_dir = 'model'

In [16]:
log = 'Adam_batch_size={}_epoch={}'.format(str(batch_size), str(num_epochs))
writer = SummaryWriter(log_dir='log/' + log)


model = Model(input_size, hidden_size, num_layers, num_classes).to(device)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [17]:
for step, (seq, label) in enumerate(dataloader):
  print(step)
  print(seq.shape)
  print(label.shape)
  break

0
torch.Size([2048, 10, 28])
torch.Size([2048, 28])


In [18]:
# Train the model
start_time = time.time()
total_step = len(dataloader)
for epoch in range(num_epochs):  # Loop over the dataset multiple times
    train_loss = 0
    for step, (seq, label) in enumerate(dataloader):
        # Forward pass
        seq = seq.clone().detach().view(-1, window_size, input_size).to(device)
        output = model(seq)
        loss = criterion(output, label.to(device))

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    print('Epoch [{}/{}], train_loss: {:.4f}'.format(epoch + 1, num_epochs, train_loss / total_step))
elapsed_time = time.time() - start_time
print('elapsed_time: {:.3f}s'.format(elapsed_time))
print('Finished Training')

Epoch [1/300], train_loss: 3.0062
Epoch [2/300], train_loss: 1.9114
Epoch [3/300], train_loss: 1.7414
Epoch [4/300], train_loss: 1.4911
Epoch [5/300], train_loss: 1.1890
Epoch [6/300], train_loss: 0.9568
Epoch [7/300], train_loss: 0.8476
Epoch [8/300], train_loss: 0.7954
Epoch [9/300], train_loss: 0.7595
Epoch [10/300], train_loss: 0.7283
Epoch [11/300], train_loss: 0.6963
Epoch [12/300], train_loss: 0.6622
Epoch [13/300], train_loss: 0.6288
Epoch [14/300], train_loss: 0.5895
Epoch [15/300], train_loss: 0.5506
Epoch [16/300], train_loss: 0.5248
Epoch [17/300], train_loss: 0.5073
Epoch [18/300], train_loss: 0.4837
Epoch [19/300], train_loss: 0.4607
Epoch [20/300], train_loss: 0.4424
Epoch [21/300], train_loss: 0.4264
Epoch [22/300], train_loss: 0.4147
Epoch [23/300], train_loss: 0.4045
Epoch [24/300], train_loss: 0.3981
Epoch [25/300], train_loss: 0.3923
Epoch [26/300], train_loss: 0.3871
Epoch [27/300], train_loss: 0.3830
Epoch [28/300], train_loss: 0.3755
Epoch [29/300], train_loss: 0

In [13]:
#save the model

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

torch.save(model.state_dict(), '/content/drive/MyDrive/LSTM_model_parameter')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


NameError: ignored

In [25]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

Model's state_dict:
lstm.weight_ih_l0 	 torch.Size([256, 28])
lstm.weight_hh_l0 	 torch.Size([256, 64])
lstm.bias_ih_l0 	 torch.Size([256])
lstm.bias_hh_l0 	 torch.Size([256])
lstm.weight_ih_l1 	 torch.Size([256, 64])
lstm.weight_hh_l1 	 torch.Size([256, 64])
lstm.bias_ih_l1 	 torch.Size([256])
lstm.bias_hh_l1 	 torch.Size([256])
fc.weight 	 torch.Size([28, 64])
fc.bias 	 torch.Size([28])


In [19]:
x,y = dataset[0]
print(x)
print(y)

tensor([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
  

In [20]:
import numpy as np

model.eval()
with torch.no_grad():
  output = model(x.view(-1, 10, 28).to(device))

print(output)
print(np.argmax(output.cpu()))

tensor([[-4.2119, -0.9358, -0.4673, -2.2237, -8.2818, -3.1104, -4.5761, -4.5197,
          0.3511, -4.4162,  1.6622, -4.6405, -4.5563, -4.3275, -4.1238, -7.0442,
         -4.7698, -2.1374, -4.2913, -4.2192, -0.0477, -2.4882, -0.7021, -4.1183,
         -1.7215, 12.4460, -4.2609, -4.2010]], device='cuda:0')
tensor(25)


In [21]:
x,y = dataset[5]
print(x)
print(y)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
  

In [22]:
model.eval()
with torch.no_grad():
  output = model(x.view(-1, 10, 28).to(device))

print(output)
print(np.argmax(output.cpu()))

tensor([[ -6.3857,   1.7995,   1.1668,  -0.4470,  -0.6188,  -3.4613,  -6.9188,
          -6.1911, -10.7469,  -6.1307,  -3.6821,  -5.9067,  -5.9824,  -6.3436,
          -5.5927,  -8.6709,  -7.1039,   0.2851,  -5.9360,  -6.4386,   1.8577,
          -3.6044,  12.1136,  -5.7297,   0.6888,  -0.5130,  -6.1667,  -5.8830]],
       device='cuda:0')
tensor(22)


PART 2

In [5]:

n_templates = 28 #total number of unique templates

def template_to_tensor(template):
    tensor = [0] * n_templates
    if template != -1 :    # we put template= -1 for session less than windows_size in empty part and it will be all zero
      tensor[template] = 1
    return tensor

In [6]:
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(Model, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))  # out.shape : [batch_size, sequence_length, hidden_size]
        out = self.fc(out[:, -1, :]) #The : before , -1, : indicates that we want to include all elements along the first dimension (batch dimension). -1 represents the index of the last element along the second dimension (sequence length). : after , -1 indicates that we want to include all elements along the third dimension (hidden size)
        return out

In [7]:
window_size = 10
num_classes = 28
input_size = 28
num_layers = 2
hidden_size = 64
num_candidates = 9 # on paper is g , top-g(here top 9) probabilities to appear next are considered normal
model = Model(input_size, hidden_size, num_layers, num_classes).to(device)

In [8]:
# upload the model

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

model_path = '/content/drive/MyDrive/LSTM_model_parameter'
model.load_state_dict(torch.load(model_path))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<All keys matched successfully>

In [9]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

Model's state_dict:
lstm.weight_ih_l0 	 torch.Size([256, 28])
lstm.weight_hh_l0 	 torch.Size([256, 64])
lstm.bias_ih_l0 	 torch.Size([256])
lstm.bias_hh_l0 	 torch.Size([256])
lstm.weight_ih_l1 	 torch.Size([256, 64])
lstm.weight_hh_l1 	 torch.Size([256, 64])
lstm.bias_ih_l1 	 torch.Size([256])
lstm.bias_hh_l1 	 torch.Size([256])
fc.weight 	 torch.Size([28, 64])
fc.bias 	 torch.Size([28])


In [10]:
row = '2 3 4 5 6 7'
line = [template_to_tensor(int(i) - 1) for i in row.strip().split()]
line = line + [[0]*28] * (window_size + 1 - len(line))
len(line)

11

In [ ]:
line = [int(i) - 1 for i in row.strip().split()]
line = line + [-1] * (window_size + 1 - len(line))
line = [template_to_tensor(template) for template in line]
line


In [14]:
def generate(name):
    # If you what to replicate the DeepLog paper results(Actually, I have a better result than DeepLog paper results),
    # you should use the 'list' not 'set' to obtain the full dataset, I use 'set' just for test and acceleration.
    hdfs = set()
    # hdfs = []
    with open('/content/' + name, 'r') as f:
        for row in f:
            line = [int(i) - 1 for i in row.strip().split()]
            line = line + [-1] * (window_size + 1 - len(line)) #if the length of the line is less than windows size, it covers by -1
            hdfs.add(tuple(line))
            # hdfs.append(tuple(line))
    print('Number of sessions({}): {}'.format(name, len(hdfs)))
    return hdfs

In [15]:
test_normal_loader = generate('hdfs_test_normal')
test_abnormal_loader = generate('hdfs_test_abnormal')

Number of sessions(hdfs_test_normal): 14177
Number of sessions(hdfs_test_abnormal): 4123


In [23]:
# Test the model

model.eval()

TP = 0
FP = 0
start_time = time.time()
with torch.no_grad():
    for line in test_normal_loader:
        for i in range(len(line) - window_size):
            session = line[i:i + window_size]
            seq = [template_to_tensor(temp) for temp in session]
            print(seq)

            label = template_to_tensor(line[i + window_size])
            print(label)
            seq = torch.tensor(seq, dtype=torch.float).view(-1, window_size, input_size).to(device)
            print(seq.shape)
            label = torch.tensor(label).view(-1).to(device)
            print(label.shape)
            output = model(seq)
            print(output)
            predicted = torch.argsort(output, 1)[0][num_candidates:]
            break
        break

[[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
torch.Size([1, 10, 28])
torch.Size([28])
tensor([[-4.3

In [31]:
print(np.argmax(label.cpu()))
print(np.argmax(output.cpu()))
print(output[0,10])



tensor(10)
tensor(10)
tensor(11.7076, device='cuda:0')


In [ ]:
session = line[i:i + window_size]
            seq = [template_to_tensor(temp) for temp in session]
            label = template_to_tensor(line[i + window_size])

In [42]:
# Test the model
TP = 0
FP = 0

start_time = time.time()
with torch.no_grad():
    for line in test_normal_loader:
        for i in range(len(line) - window_size):
            session = line[i:i + window_size]
            seq = [template_to_tensor(temp) for temp in session]
            label = template_to_tensor(line[i + window_size])
            seq = torch.tensor(seq, dtype=torch.float).view(-1, window_size, input_size).to(device)
            label = torch.tensor(label).view(-1).to(device)
            output = model(seq)
            predicted = torch.argsort(output, 1)[0][-num_candidates:]
            if torch.argmax(label) not in predicted:
                FP += 1
                break   #with just one wrong prediction in a line , we assume , abnormal
with torch.no_grad():
    for line in test_abnormal_loader:
        for i in range(len(line) - window_size):
            session = line[i:i + window_size]
            seq = [template_to_tensor(temp) for temp in session]
            label = template_to_tensor(line[i + window_size])
            seq = torch.tensor(seq, dtype=torch.float).view(-1, window_size, input_size).to(device)
            label = torch.tensor(label).view(-1).to(device)
            output = model(seq)
            predicted = torch.argsort(output, 1)[0][-num_candidates:]
            if torch.argmax(label) not in predicted:
                TP += 1
                break
elapsed_time = time.time() - start_time
print('elapsed_time: {:.3f}s'.format(elapsed_time))
# Compute precision, recall and F1-measure
FN = len(test_abnormal_loader) - TP
P = 100 * TP / (TP + FP)
R = 100 * TP / (TP + FN)
F1 = 2 * P * R / (P + R)
print('false positive (FP): {}, false negative (FN): {}, Precision: {:.3f}%, Recall: {:.3f}%, F1-measure: {:.3f}%'.format(FP, FN, P, R, F1))
print('Finished Predicting')

elapsed_time: 180.065s
false positive (FP): 350, false negative (FN): 744, Precision: 90.614%, Recall: 81.955%, F1-measure: 86.067%
Finished Predicting
